In [ ]:
! nvidia-smi

Tue Aug 23 01:37:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os

class Config:
    AUTHOR = "wanwan7123"

    NAME = "feedback-Exp042-essay-deberta-large"
    MODEL_PATH = "microsoft/deberta-large"
    DATASET_PATH = []

    COMPETITION = "feedback-prize-effectiveness"
    COLAB_PATH = "/content/drive/MyDrive/DataAnalysis/competicion/competicion_feedback" 
    DRIVE_PATH = os.path.join(COLAB_PATH, AUTHOR)

    api_path = "/content/drive/MyDrive/kaggle.json"

    seed = 42
    num_fold = 5
    trn_fold = [0, 1, 2, 3, 4]
    batch_size = 4
    n_epochs = 5
    
    fc_dropout = 0.1
    weight_decay = 0.001
    beta = (0.9, 0.98)
    lr = 5e-6
    eval_steps = 499
    num_warmup_steps_rate = 0.01
    clip_grad_norm = None
    gradient_accumulation_steps = 1
    
    # GPU Optimize Settings
    gpu_optimize_config= {
        "fp16": True,
        "freezing": True,
        "optim8bit": True,
        "gradient_checkpoint": True
    }

    upload_from_colab = True

In [ ]:
import os
import re
import gc
import sys
import json
import time
import shutil
import joblib
import random
import requests
import warnings
warnings.filterwarnings('ignore')
from ast import literal_eval
from tqdm import tqdm
from pathlib import Path
from glob import glob

import numpy as np
import pandas as pd
import scipy 
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import (
    StratifiedKFold, 
    KFold, 
    GroupKFold,
    StratifiedGroupKFold
)
from sklearn.metrics import log_loss
!pip install torch==1.10

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.checkpoint import checkpoint
from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils.rnn import pad_sequence

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def setup(cfg):
    cfg.COLAB = 'google.colab' in sys.modules
    cfg.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if cfg.COLAB:
        print('This environment is Google Colab')

        # mount
        from google.colab import drive
        if not os.path.isdir('/content/drive'):
            drive.mount('/content/drive') 

        # pip install
        ! pip install transformers==4.16.2
        ! pip install tokenizers==0.11.6
        ! pip install transformers[sentencepiece]

        # use kaggle api (need kaggle token)
        f = open(cfg.api_path, 'r')
        json_data = json.load(f) 
        os.environ['KAGGLE_USERNAME'] = json_data['username']
        os.environ['KAGGLE_KEY'] = json_data['key']

        # set dirs
        cfg.DRIVE = cfg.DRIVE_PATH
        cfg.EXP = (cfg.NAME if cfg.NAME is not None 
            else requests.get('http://172.28.0.2:9000/api/sessions').json()[0]['name'][:-6]
        )
        cfg.INPUT = os.path.join(cfg.DRIVE, 'Input')
        cfg.OUTPUT = os.path.join(cfg.DRIVE, 'Output')
        cfg.SUBMISSION = os.path.join(cfg.DRIVE, 'Submission')
        cfg.DATASET = os.path.join(cfg.DRIVE, 'Dataset')

        cfg.OUTPUT_EXP = os.path.join(cfg.OUTPUT, cfg.EXP) 
        cfg.EXP_MODEL = os.path.join(cfg.OUTPUT_EXP, 'model')
        cfg.EXP_FIG = os.path.join(cfg.OUTPUT_EXP, 'fig')
        cfg.EXP_PREDS = os.path.join(cfg.OUTPUT_EXP, 'preds')

        # make dirs
        for d in [cfg.INPUT, cfg.SUBMISSION, cfg.EXP_MODEL, cfg.EXP_FIG, cfg.EXP_PREDS]:
            os.makedirs(d, exist_ok=True)
        
        if not os.path.isfile(os.path.join(cfg.INPUT, 'train.csv')):
            # load dataset
            ! pip install --upgrade --force-reinstall --no-deps kaggle
            ! kaggle competitions download -c $cfg.COMPETITION -p $cfg.INPUT
            filepath = os.path.join(cfg.INPUT,cfg.COMPETITION+'.zip')
            ! unzip -d $cfg.INPUT $filepath
            
        
        for path in cfg.DATASET_PATH:
            datasetpath = os.path.join(cfg.DATASET,  path.split('/')[1])
            if not os.path.exists(datasetpath):
                os.makedirs(datasetpath, exist_ok=True)
                ! kaggle datasets download $path -p $datasetpath
                filepath = os.path.join(datasetpath, path.split("/")[1]+'.zip')
                ! unzip -d $datasetpath $filepath

    else:
        print('This environment is Kaggle Kernel')

        # set dirs
        cfg.INPUT = f'../input/{cfg.COMPETITION}'
        cfg.EXP = cfg.NAME
        cfg.OUTPUT_EXP = cfg.NAME
        cfg.SUBMISSION = './'
        cfg.DATASET = '../input/'
        
        cfg.EXP_MODEL = os.path.join(cfg.EXP, 'model')
        cfg.EXP_FIG = os.path.join(cfg.EXP, 'fig')
        cfg.EXP_PREDS = os.path.join(cfg.EXP, 'preds')

        # make dirs
        for d in [cfg.EXP_MODEL, cfg.EXP_FIG, cfg.EXP_PREDS]:
            os.makedirs(d, exist_ok=True)
    return cfg


def dataset_create_new(dataset_name, upload_dir):
    dataset_metadata = {}
    dataset_metadata['id'] = f'{os.environ["KAGGLE_USERNAME"]}/{dataset_name}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = dataset_name
    with open(os.path.join(upload_dir, 'dataset-metadata.json'), 'w') as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode='tar')

In [ ]:
# =====================
# Utils
# =====================
# Seed
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# KFold
def get_kfold(train, n_splits, seed):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    generator = kf.split(train)
    fold_series = []
    for fold, (idx_train, idx_valid) in enumerate(generator):
        fold_series.append(pd.Series(fold, index=idx_valid))
    fold_series = pd.concat(fold_series).sort_index()
    return fold_series

def get_stratifiedkfold(train, target_col, n_splits, seed):
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    generator = kf.split(train, train[target_col])
    fold_series = []
    for fold, (idx_train, idx_valid) in enumerate(generator):
        fold_series.append(pd.Series(fold, index=idx_valid))
    fold_series = pd.concat(fold_series).sort_index()
    return fold_series

def get_groupkfold(train, target_col, group_col, n_splits):
    kf = GroupKFold(n_splits=n_splits)
    generator = kf.split(train, train[target_col], train[group_col])
    fold_series = []
    for fold, (idx_train, idx_valid) in enumerate(generator):
        fold_series.append(pd.Series(fold, index=idx_valid))
    fold_series = pd.concat(fold_series).sort_index()
    return fold_series

def get_groupstratifiedkfold(train, target_col, group_col, n_splits, seed):
    kf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    generator = kf.split(train, train[target_col], train[group_col])
    fold_series = []
    for fold, (idx_train, idx_valid) in enumerate(generator):
        fold_series.append(pd.Series(fold, index=idx_valid))
    fold_series = pd.concat(fold_series).sort_index()
    train['fold'] = fold_series
    return train, fold_series

In [ ]:
# 文章のバグを治す
from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs

def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

# Register the encoding and decoding error handlers for `utf-8` and `cp1252`.
codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    """Resolve the encoding problems and normalize the abnormal characters."""
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text

In [ ]:
def flatten(_list):
    return list(itertools.chain.from_iterable(_list))

def even_split(input_ids):
    best_idx = None
    best_len = 100000
    for i in range(1, len(input_ids)):
        x_len = len(flatten(input_ids[:i]))
        y_len = len(flatten(input_ids[i:]))
        diff = abs(x_len - y_len)
        
        if best_len > diff:
            best_len = diff
            best_idx = i
    
    return best_idx

def preprocess_df(df, tokenizer, max_length=198, total_max_length:int=1024):
    df['discourse_text'] = df['discourse_text'].apply(lambda x : resolve_encodings_and_normalize(x))
    df['discourse_start'] = df['discourse_type'].apply(lambda x: f'<{x} start>')
    df['discourse_end'] = df['discourse_type'].apply(lambda x: f'<{x} end>')
    df['input_text'] = df['discourse_start'] + " " + df['discourse_text'] + " " + df['discourse_end']

    # one-hot型の準備
    label_ar = df['label'].values
    onehot_ar = np.eye(3)[label_ar] 
    df['Ineffective'] = onehot_ar[:, 0]
    df['Adequate'] = onehot_ar[:, 1]
    df['Effective'] = onehot_ar[:, 2]
    df['label_list'] = df[['Ineffective', 'Adequate', 'Effective']].values.tolist()


    gdf = df.groupby("essay_id")
    fold_df = df.groupby('essay_id')['fold'].apply(lambda x: list(x)[0])
    
    essay_inputs = df.groupby("essay_id")["input_text"].apply(list)
    essay_ids = essay_inputs.index.tolist()
    
    labels = gdf["label_list"].apply(list)
    discourse_ids = gdf["discourse_id"].apply(list)
    
    rows = []
    for i in tqdm(range(len(essay_inputs))):
        # まず全体をtokenizeして1024に収まっていれば、各テキストをtruncationしておく必要はない
        input_ids = tokenizer.batch_encode_plus(essay_inputs[i], max_length=total_max_length, truncation=True)["input_ids"]
        
        if len(flatten(input_ids)) > total_max_length:
            split_idx = even_split(input_ids)
            
            first = input_ids[:split_idx]
            first_seq_ids = [[seq_ids]*len(ids) for seq_ids, ids in enumerate(first)]

            second = input_ids[split_idx:]
            second_seq_ids = [[seq_ids]*len(ids) for seq_ids, ids in enumerate(second)]
            essay_id = essay_ids[i]
            
            rows.append({
                "essay_id":essay_ids[i],
                "group":1,
                "discourse_id": discourse_ids[i][:split_idx ],
                "label":labels[i][:split_idx],
                "input_ids":flatten(first),
                "seq_ids":flatten(first_seq_ids),
                "fold":fold_df[essay_id],
            })
        
            rows.append({
                "essay_id":essay_ids[i],
                "group":2,
                "discourse_id": discourse_ids[i][split_idx:],
                "label":labels[i][split_idx:],
                "input_ids":flatten(second),
                "seq_ids":flatten(second_seq_ids),
                "fold":fold_df[essay_id],
            })
        else:
            # もしかしたら一つのtextで1024超える場合もある？
            seq_ids = [[seq_ids]*len(ids) for seq_ids, ids in enumerate(input_ids)]
            input_ids = flatten(input_ids)
            essay_id = essay_ids[i]
            
            rows.append({
                "essay_id":essay_ids[i],
                "group":1,
                "discourse_id": discourse_ids[i],
                "label":labels[i],
                "input_ids":input_ids,
                "seq_ids":flatten(seq_ids),
                "fold":fold_df[essay_id],
            })
            
    return pd.DataFrame(rows)

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, df, tokenizer):
        super().__init__()
        self.df = df
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        
        inputs = {}
        
        input_ids = np.array(row["input_ids"])
        seq_ids = np.array(row["seq_ids"])
        label = np.array(row['label'])
        
        attention_mask = np.array([1 if id != self.tokenizer.pad_token_id else 0 for id in input_ids], dtype=np.int64)
        
        inputs = {
            "discourse_id":row["discourse_id"],
            "input_ids":input_ids,
            "attention_mask":attention_mask,
            "seq_ids":seq_ids,
            "label":label,
        }
        
        return inputs

# バッチごとにパディング操作を行う
class Collator:
    def __init__(self, tokenizer, input_cols, meta_cols=None):
        self.tokenizer = tokenizer
        self.input_cols = input_cols
        
        self.meta_cols = meta_cols if meta_cols is not None else []
        
        self.padding = True
        self.max_length: Optional[int] = None
        self.pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        first = features[0]
        
        input_features = []
        meta_features = {meta_col:[] for meta_col in self.meta_cols}
        
        for f in features:
            input_features.append({col:f[col] for col in self.input_cols})
            for meta_col in self.meta_cols:
                meta_features[meta_col].extend(f[meta_col])
            
        batch = self.tokenizer.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        if "label" in first:
            batch["labels"] = pad_sequence([torch.tensor(f["label"], dtype=torch.float) for f in features], batch_first=True, padding_value=-1)

        if "seq_ids" in first:
            batch["seq_ids"] = pad_sequence([torch.tensor(f["seq_ids"], dtype=torch.long) for f in features], batch_first=True, padding_value=-1)
            
        if self.meta_cols is not None:
            batch["meta"] = meta_features

        
        return batch

In [ ]:
class SimpleHeader(nn.Module):
    def __init__(self, input_size:int, num_labels):
        super().__init__()
        self.fc = nn.Linear(input_size, num_labels)

        self.cls_dropouts = nn.Sequential(
            nn.Dropout(0.1),
            nn.Dropout(0.2),
            nn.Dropout(0.3),
            nn.Dropout(0.4),
            nn.Dropout(0.5),
        )

    def forward(self, x):
        output = torch.mean(
            torch.stack(
                [self.fc(self.cls_dropouts[i](x)) for i in range(5)],
                dim=0,
            ),
            dim=0,
        )
        return output

class LitModel(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.cfg = cfg
        self.gpu_optimize_config = cfg.gpu_optimize_config
        self.config = AutoConfig.from_pretrained(
            cfg.MODEL_PATH,
            output_hidden_states=True
        )
        self.config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": 0.1,
                "layer_norm_eps": 1e-7,
                "add_pooling_layer": False,
                "num_labels": 3,
            }
        )
        self.backbone = AutoModel.from_pretrained(
            cfg.MODEL_PATH,
            config=self.config
        )   
        self.hidden_size = self.config.hidden_size
        self.fc = nn.Linear(self.config.hidden_size, 3)
        self.header = SimpleHeader(input_size=self.config.hidden_size, num_labels=3)
        self._init_weights(self.header.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=0)
        )
        self._init_weights(self.attention)

        # Gradient Checkpointing
        if self.gpu_optimize_config['gradient_checkpoint']:
            self.backbone.gradient_checkpointing_enable()

    @property
    def device(self):
        return self.backbone.device
            
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, last_hidden_state):
        weights = self.attention(last_hidden_state)
        feature = torch.sum(weights * last_hidden_state, dim=0)
        return feature

    # トークンごとに平均取って潰す
    def sequence_mean(self, logits, batch):
        batch_seq_mean = []
        batch_size = batch["input_ids"].shape[0]
        seq_lens = []
        # バッチサイズごとに
        for i in range(batch_size):
            seq_mean = []
            # バッチの長さに応じて処理
            # iはバッチを表すので、そのバッチで何個文章がくっついてるかをjは表す
            for j in range(max(batch["seq_ids"][i])+1):
                # i, j成分（該当するdiscourse）を取り出す
                idx = batch["seq_ids"][i]==j
                idx = idx.nonzero().reshape(-1)
                # idxでtensorの抜き出しを行う
                seq_tensor = torch.index_select(logits[i], 0, idx)
                seq_tensor = self.feature(seq_tensor)
                seq_mean.append(seq_tensor)

            seq_lens.append(len(seq_mean))
            batch_seq_mean.append(torch.vstack(seq_mean))

        return batch_seq_mean, seq_lens
            
    def forward(self, input_dict, labels):
        # batch, len, hidden_size
        output = self.backbone(
            input_ids=input_dict["input_ids"],
            attention_mask=input_dict["attention_mask"]
        )["last_hidden_state"]
    
        # discourse, hidden_size
        output, seq_lens = self.sequence_mean(output, input_dict)
        # discourse, hidden_size
        output = torch.vstack(output)
        output = self.header(output)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(output, labels)
            return loss, output
        else:
            return output

In [ ]:
# FGM
# https://www.kaggle.com/competitions/tweet-sentiment-extraction/discussion/143764#809408

class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=1.0, emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0:
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
            self.backup = {}

In [ ]:
def training(cfg, train):
    # =====================
    # Training
    # =====================
    set_seed(cfg.seed)
    oof_all_df = pd.DataFrame()
    for fold in cfg.trn_fold:
        # dataset, dataloader
        train_df = train.loc[train['fold']!=fold]
        valid_df = train.loc[train['fold']==fold]

        # psuedo_dataの追加
        psuedo_df = pd.read_csv(f'/content/drive/MyDrive/DataAnalysis/competicion/competicion_feedback/wanwan7123/Input/zaccheaus_exp016/zac_exp016_psuedo_sample_fold{fold}.csv')
        psuedo_df['discourse_id'] = psuedo_df['discourse_id'].apply(lambda x: eval(str(x)))
        psuedo_df['label'] = psuedo_df['label'].apply(lambda x: eval(str(x)))
        psuedo_df['input_ids'] = psuedo_df['input_ids'].apply(lambda x: eval(str(x)))
        psuedo_df['seq_ids'] = psuedo_df['seq_ids'].apply(lambda x: eval(str(x)))
        train_df = pd.concat([train_df, psuedo_df]).reset_index(drop=True)

        train_idx = list(train_df.index)
        valid_idx = list(valid_df.index)

        # Datasetの設定
        train_dataset = TrainDataset(train_df, cfg.tokenizer)
        valid_dataset = TrainDataset(valid_df, cfg.tokenizer)
        train_loader = DataLoader(
            dataset=train_dataset, 
            batch_size=cfg.batch_size, 
            shuffle=True,
            pin_memory=True,
            drop_last=True,
            collate_fn = Collator(cfg.tokenizer, input_cols = ["input_ids", "attention_mask"], meta_cols = ["discourse_id"])
        )
        valid_loader = DataLoader(
            dataset=valid_dataset,
            batch_size=cfg.batch_size,
            shuffle=False,
            pin_memory=True,
            drop_last=False,
            collate_fn = Collator(cfg.tokenizer, input_cols = ["input_ids", "attention_mask"], meta_cols = ["discourse_id"])
        )

        # model
        model = LitModel(cfg)
        torch.save(model.config, cfg.EXP_MODEL+'config.pth')
        model = model.to(cfg.device)

        # optimizer, scheduler
        param_optimizer = list(model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {
                'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                'weight_decay': cfg.weight_decay
            },
            {
                'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                'weight_decay': 0.0
            }
        ]

        # optimizer
        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=cfg.lr,
            betas=cfg.beta,
            weight_decay=cfg.weight_decay,
        )
        
        # scaler
        scaler = GradScaler()

        # enable FGM
        fgm = FGM(model)

        num_train_optimization_steps = int(
            len(train_loader) * cfg.n_epochs // cfg.gradient_accumulation_steps
        )
        num_warmup_steps = int(num_train_optimization_steps * cfg.num_warmup_steps_rate)
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=num_train_optimization_steps
        )

        # model-training
        criterion = nn.CrossEntropyLoss()
        best_val_score = 9999
        
        for epoch in range(cfg.n_epochs):
            # training
            print(f"# ============ start epoch:{epoch} ============== #")
            train_losses = []
            train_nums = []
            model.train() 
            val_losses_batch = []
            ids = []
            # dataloader回して予測
            with tqdm(train_loader, total=len(train_loader)) as pbar:
                for step, (inputs) in enumerate(pbar):

                    meta = inputs.pop("meta", None)
                    inputs = {k:v.to(cfg.device) for k, v in inputs.items()}
                    labels = inputs['labels'][torch.where(inputs['labels'] != -1)].reshape(-1, 3)
                    optimizer.zero_grad()

                    with autocast():
                        loss, output = model(inputs, labels)
                    
                    ids.extend(meta["discourse_id"])
                    pbar.set_postfix({
                        'loss': loss.item(),
                        'lr': scheduler.get_lr()[0]
                    })
                    train_losses.append(loss.item() * len(labels))
                    train_nums.append(len(labels))

                    if cfg.gradient_accumulation_steps > 1:
                        loss = loss / cfg.gradient_accumulation_steps

                    scaler.scale(loss).backward()

                    # FGM attack
                    fgm.attack()
                    with autocast():
                        loss_adv, _ = model(inputs, labels)
                    scaler.scale(loss_adv).backward()
                    fgm.restore()
                    
                    if cfg.clip_grad_norm is not None:
                        torch.nn.utils.clip_grad_norm_(
                            model.parameters(), 
                            cfg.clip_grad_norm
                        )
                    if (step+1) % cfg.gradient_accumulation_steps == 0:
                        scaler.step(optimizer)
                        scaler.update()
                        scheduler.step()

                    # evaluating
                    if ((step+1) % cfg.eval_steps == 0):
                        print('===========steps：', step, '==========')
                        val_preds = []
                        val_losses = []
                        val_nums = []
                        ids = []
                        model.eval()
                        with torch.no_grad():
                            with tqdm(valid_loader, total=len(valid_loader)) as pbar:
                                for steps, (inputs) in enumerate(pbar):

                                    meta = inputs.pop("meta", None)
                                    inputs = {k:v.to(cfg.device) for k, v in inputs.items()}
                                    labels = inputs['labels'][torch.where(inputs['labels'] != -1)].reshape(-1, 3)

                                    with autocast():
                                        loss, output = model(inputs, labels)
                                    ids.extend(meta["discourse_id"])

                                    output = output.detach().cpu().numpy()
                                    val_preds.append(output)
                                    val_losses.append(loss.item() * len(labels))
                                    val_nums.append(len(labels))
                                    pbar.set_postfix({
                                        'val_loss': loss.item()
                                    })

                        val_preds = np.vstack(val_preds)
                        val_loss = sum(val_losses) / sum(val_nums)

                        val_log = {
                            'val_loss': val_loss
                        }
                        display(val_log)

                        if best_val_score > val_loss:
                            print("save model weight")
                            best_val_preds = val_preds
                            best_val_score = val_loss
                            torch.save(
                                model.state_dict(), 
                                os.path.join(cfg.EXP_MODEL, f"fold{fold}.pth")
                            )
                        
                        model.train()

            train_loss = sum(train_losses)/sum(train_nums)
            train_log = {
                'train_loss':train_loss
            }
            display(train_log)

            # evaluating(per epoch)
            print(' ==========end epoch ==========')
            val_preds = []
            val_losses = []
            val_nums = []
            ids = []
            model.eval()
            with torch.no_grad():
                with tqdm(valid_loader, total=len(valid_loader)) as pbar:
                    for steps, (inputs) in enumerate(pbar):

                        meta = inputs.pop("meta", None)
                        inputs = {k:v.to(cfg.device) for k, v in inputs.items()}
                        labels = inputs['labels'][torch.where(inputs['labels'] != -1)].reshape(-1, 3)

                        with autocast():
                            loss, output = model(inputs, labels)
                        ids.extend(meta["discourse_id"])

                        output = output.detach().cpu().numpy()
                        val_preds.append(output)
                        val_losses.append(loss.item() * len(labels))
                        val_nums.append(len(labels))
                        pbar.set_postfix({
                            'val_loss': loss.item()
                        })

            val_preds = np.vstack(val_preds)
            val_loss = sum(val_losses) / sum(val_nums)

            val_log = {
                'val_loss': val_loss
            }
            display(val_log)

            if best_val_score > val_loss:
                print("save model weight")
                best_val_preds = val_preds
                best_val_score = val_loss
                torch.save(
                    model.state_dict(), 
                    os.path.join(cfg.EXP_MODEL, f"fold{fold}.pth")
                )

        oof_df = pd.DataFrame(ids, columns=['discourse_id'])
        oof_df['Ineffective'] = best_val_preds[:, 0]
        oof_df['Adequate'] = best_val_preds[:, 1]
        oof_df['Effective'] = best_val_preds[:, 2]
        oof_df.to_csv(os.path.join(cfg.EXP_PREDS, f'oof_pred_fold{fold}.csv'))
        oof_all_df = pd.concat([oof_all_df, oof_df], axis=0)
        del model; gc.collect()

    oof_all_df.to_csv(os.path.join(cfg.EXP_PREDS, 'oof_pred.csv'))

    # =====================
    # scoring
    # =====================
    '''
    metric = nn.CrossEntropyLoss()
    score = metric(torch.from_numpy(oof_pred), torch.from_numpy(train['label'].values))
    print('CV:', score.to('cpu').detach().numpy())
    '''
    return oof_all_df

In [ ]:
# =====================
# Main
# =====================

# setup
cfg = setup(Config)

import transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer
from transformers import AdamW, get_cosine_schedule_with_warmup
import tokenizers
import sentencepiece
%env TOKENIZERS_PARALLELISM=true
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")

# main
train = pd.read_csv(os.path.join(cfg.INPUT, 'train_full.csv'))
test = pd.read_csv(os.path.join(cfg.INPUT, 'test.csv'))
sub = pd.read_csv(os.path.join(cfg.INPUT, 'sample_submission.csv'))

cfg.tokenizer = AutoTokenizer.from_pretrained(cfg.MODEL_PATH)
cfg.tokenizer.save_pretrained(os.path.join(cfg.OUTPUT_EXP, 'tokenizer'))
train['label'] = train['discourse_effectiveness'].map({'Ineffective':0, 'Adequate':1, 'Effective':2})
train['type_label'] = train['discourse_type'] + '_' + train['discourse_effectiveness']
train, cfg.folds = get_groupstratifiedkfold(train, 'type_label', 'essay_id', cfg.num_fold, cfg.seed)
cfg.folds.to_csv(os.path.join(cfg.EXP_PREDS, 'folds.csv'))

train = preprocess_df(train, cfg.tokenizer, max_length=198, total_max_length=1024)
display(train.head())
score = training(cfg, train)

if cfg.upload_from_colab and cfg.COLAB:
    from kaggle.api.kaggle_api_extended import KaggleApi
    dataset_create_new(dataset_name=Config.EXP, upload_dir=Config.OUTPUT_EXP)

This environment is Google Colab
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
env: TOKENIZERS_PARALLELISM=true
tokenizers.__version__: 0.11.6
transformers.__version__: 4.16.2


100%|██████████| 4191/4191 [00:06<00:00, 643.49it/s]


,essay_id,group,discourse_id,label,input_ids,seq_ids,fold
0,00066EA9880D,1,"[fe6dfbd53216, ca9e1b60c9fb, 6cf2157f4f19, d92...","[[0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [0.0, 0.0, ...","[1, 41552, 32258, 386, 15698, 16870, 1672, 167...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
1,000E6DE9E817,1,"[695d181861a1, cd97ee1cc0ad, 1b775274990b, 567...","[[0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, ...","[1, 41552, 46884, 386, 15698, 38, 524, 7594, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
2,0016926B079C,1,"[89304284cef1, 4f2e871a4908, a885c3aa214b, 953...","[[0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, ...","[1, 41552, 46884, 386, 15698, 38, 206, 14, 521...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
3,00203C45FC55,1,"[a713d0f6dc68, 2fd9bb2bfedf, 0e5ecdf1516e, 499...","[[0.0, 0.0, 1.0], [0.0, 0.0, 1.0], [0.0, 0.0, ...","[1, 41552, 32258, 386, 15698, 85, 16, 358, 129...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,00203C45FC55,2,"[44458755d833, e0be5aef7bed, f8a81ae083d4, 248...","[[0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [0.0, 0.0, ...","[1, 41552, 44057, 31628, 386, 15698, 635, 6, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


Some weights of the model checkpoint at microsoft/deberta-large were not used when initializing DebertaModel: ['lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# ============ start epoch:0 ============== #


...

{'val_loss': 0.6189149831244454}

save model weight
# ============ start epoch:2 ============== #


 49%|████▊     | 498/1026 [1:50:28<1:56:20, 13.22s/it, loss=0.707, lr=2.56e-6]

===========steps： 498 ==========



 50%|█████     | 113/224 [22:35<23:21, 12.62s/it, val_loss=0.71]